In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,583 kB]
Get:14 http://security.ubuntu.com/ubuntu j

In [2]:
!google-chrome --version

/bin/bash: line 1: google-chrome: command not found


In [3]:
pip install requests -q  # 處理 HTTP 請求

In [4]:
pip install Selenium -q  # 模擬瀏覽器行為，進行網頁自動化測試或網頁抓取

In [5]:
pip install beautifulsoup4 -q  # 解析 HTML，從中提取特定的元素和資料

In [6]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import csv

In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)

In [8]:
driver.get("https://forums.macrumors.com/whats-new/posts/4091489/")

In [9]:
all_threads_data = []

main_page_url = 'https://forums.macrumors.com/whats-new/posts/4091489/'

# print(soup)

In [ ]:
try:
    while True:
        # response = requests.get(main_page_url)  # 發送請求獲取該頁面內容

        main_page_source = driver.page_source
        print(f"Scraping data from {driver.current_url}")

        soup = BeautifulSoup(main_page_source, "lxml")

        titles = soup.select("a[data-xf-init='preview-tooltip']")
        count = 0
        for title in titles:
            if count == 100:
                break
            count += 1
            thread_link = "https://forums.macrumors.com/"+title["href"]
            print(f"Scraping thread: {thread_link}")

            # response_thread = requests.get(thread_link)   # 發送請求獲取該頁面內容

            # 使用 Selenium 獲取討論串頁面
            driver.get(thread_link)
            time.sleep(2)  # 等待頁面加載

            one_thread_content = []

            # 爬取該討論串的回覆內容
            while True:
                thread_page_source = driver.page_source
                soup_thread = BeautifulSoup(thread_page_source, "lxml")

                # 找到當前頁面所有的回覆內容
                find_thread = soup_thread.find_all("div", class_="bbWrapper")
                find_thread_time = soup_thread.find_all("time", class_="u-dt")

                for content, post_time in zip(find_thread, find_thread_time):
                    blockquote = content.find("blockquote")
                    if blockquote:
                        blockquote.decompose()  # 移除回覆中的引用區塊

                    result_discuss = content.get_text(strip=True)
                    post_time_text = post_time.get("data-date-string", "").strip()  # 抓取每條回覆的發佈時間

                    # 將每個回覆單獨加入 all_threads_data 列表
                    all_threads_data.append({
                        "thread_title": title.get_text(strip=True),
                        "content": result_discuss,
                        "time": post_time_text,
                        "link": thread_link
                    })

                # 檢查是否有下一頁
                try:
                    next_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.pageNav-jump--next"))
                    )
                    next_button.click()
                    time.sleep(2)  # 等待頁面加載
                except:
                    print("No more pages in this thread. Moving to the next.")
                    break

            # # 將當前討論串及其所有回覆加入 all_threads_data 列表
            # all_threads_data.append({
            #     "thread": thread_link,
            #     "content": "\n".join(one_thread_content)  # 把所有回覆用換行符連接起來
            # })


        # 主頁是否有更多討論串可以抓取
        try:
            next_button_main = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a.pageNav-jump--next"))
            )
            next_button_main.click()
            time.sleep(2)  # 等待頁面加載
        except:
            print("No more threads to scrape. Exiting.")
            break


finally:
    driver.quit()

Scraping data from https://forums.macrumors.com/whats-new/posts/4114453/
Scraping thread: https://forums.macrumors.com//threads/uk-2019-mac-pro-desirability.2435860/
No more pages in this thread. Moving to the next.
Scraping thread: https://forums.macrumors.com//threads/iphone-16-pro-demand-has-been-lower-than-expected-analyst-says.2436413/
No more pages in this thread. Moving to the next.
Scraping thread: https://forums.macrumors.com//threads/ios-18-battery-life-thread.2428672/
No more pages in this thread. Moving to the next.
Scraping thread: https://forums.macrumors.com//threads/apple-says-these-five-changes-make-iphone-16-models-easier-to-repair.2436885/
No more pages in this thread. Moving to the next.
Scraping thread: https://forums.macrumors.com//threads/u-s-a-apple-com-new-apple-watch-series-10-ultra-2-us-pre-orders-orders-and-delivery-status-thread-2024.2435732/
No more pages in this thread. Moving to the next.
Scraping thread: https://forums.macrumors.com//threads/where-is-ap

In [ ]:
# 將結果寫入 CSV 文件
csv_file = "threads_data.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["thread_title", "content", "time", "link"])
    writer.writeheader()
    for data in all_threads_data:
        writer.writerow(data)

print(f"Data has been saved to {csv_file}")

Data has been saved to threads_data.csv


In [ ]:
'''
main_url = 'https://forums.macrumors.com/forums/iphone.99/'

response = requests.get(main_url)
soup = BeautifulSoup(response.text, 'html.parser')
'''

In [ ]:
'''
# 找到目標 <a> 標籤
titles = soup.select("a[data-xf-init='preview-tooltip']")

for title in titles:
    thread_link = "https://forums.macrumors.com/"+title["href"]
    print(thread_link)
    response_thread = requests.get(thread_link)
    soup_thread = BeautifulSoup(response_thread.text, "lxml")
    find_thread = soup_thread.find_all("div", class_="bbWrapper")
    # print(find_thread)
    for content in find_thread:
        result_discuss = content.get_text(strip=True)
        print(result_discuss)

    break
'''